In [ ]:
# Replace with your Fyers App details
client_id = "YOUR_CLIENT_ID"       # e.g., ABC123-100
redirect_uri = "YOUR_REDIRECT_URI" # e.g., http://localhost:8000/callback
response_type = "code"

# Your login credentials
username = "your_fyers_username"
password = "your_fyers_password"
pin = "your_6_digit_pin"

# Build the login URL
auth_url = f"https://api.fyers.in/api/v2/generate-authcode?client_id={client_id}&redirect_uri={redirect_uri}&response_type={response_type}&state=sample"

# Set up Selenium
driver = webdriver.Chrome()  # You can specify the path to chromedriver if needed
driver.get(auth_url)

# Let the page load
time.sleep(2)

# Enter username
driver.find_element(By.ID, "login_email").send_keys(username)
driver.find_element(By.ID, "submitBtn").click()

time.sleep(2)

# Enter password
driver.find_element(By.ID, "login_password").send_keys(password)
driver.find_element(By.ID, "submitBtn").click()

time.sleep(2)

# Enter PIN
for i, digit in enumerate(pin):
    driver.find_element(By.ID, f"pin{i+1}").send_keys(digit)

driver.find_element(By.ID, "submitBtn").click()

# Wait for redirect
time.sleep(3)

# Capture the redirect URL
redirected_url = driver.current_url
driver.quit()

# Extract the auth_code from the URL
parsed_url = urllib.parse.urlparse(redirected_url)
auth_code = urllib.parse.parse_qs(parsed_url.query).get("auth_code", [None])[0]

print("Auth Code:", auth_code)

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import urllib.parse

In [4]:
client_id = "IHVGT4TRNZ-100"       # e.g., ABC123-100
redirect_uri = "http://127.0.0.1:8080/" # e.g., http://localhost:8000/callback
response_type = "code"

In [5]:
username = " YJ08526"
# password = "your_fyers_password"
pin = "7776"

# Build the login URL
auth_url = f"https://api.fyers.in/api/v2/generate-authcode?client_id={client_id}&redirect_uri={redirect_uri}&response_type={response_type}&state=sample"


In [ ]:
driver = webdriver.Chrome()  # You can specify the path to chromedriver if needed
driver.get(auth_url)

In [4]:
client_id = 'ETYHMVN1Q6-100'
secret_key = 'I0CJUFFA03'
redirect_uri = 'https://www.google.com/'
response_type = 'code'
state = 'sample_state'

In [3]:
from fyers_apiv3 import fyersModel

In [5]:
session = fyersModel.SessionModel(
    client_id= client_id,
    secret_key= secret_key,
    redirect_uri= redirect_uri,
    response_type= response_type
)

In [7]:
response = session.generate_authcode()